In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import seaborn as sns
import requests

import wikipedia
from urllib.parse import unquote

In [31]:
df_potential_tech1 = pd.read_excel('Datasets/Lists of Emerging Technologies and other Technologies.xlsx', sheet_name = '1. ET List (Complete N=242)')
df_potential_tech2 = pd.read_excel('Datasets/Lists of Emerging Technologies and other Technologies.xlsx', sheet_name = '2. WP Articles with Q11016 (tec')
df_potential_tech3 = pd.read_excel('Datasets/Lists of Emerging Technologies and other Technologies.xlsx', sheet_name = '3. XT List (Selected Types N=3,')

df_potential_tech1['WP_Term'] = df_potential_tech1['Emerging Tech List (n=242)'].apply(lambda x: x.split('/')[-1].replace('_',' '))
df_potential_tech2['WP_Term'] = df_potential_tech2['itemLabel']
df_potential_tech2['Wikidata_ID'] = df_potential_tech2['item'].apply(lambda x: x.split('/')[-1])
df_potential_tech3['WP Article'] = df_potential_tech3['WP Article'].fillna(df_potential_tech3['Unnamed: 1'])
df_potential_tech3['WP_Term'] = df_potential_tech3['WP Article'].apply(lambda x: x.split('/')[-1].replace('_',' '))

tech_term_list = pd.Series(df_potential_tech1['WP_Term'].tolist() + df_potential_tech3['WP_Term'].tolist()).drop_duplicates().tolist()
tech_term_list = pd.Series(tech_term_list)[~pd.Series(tech_term_list).isin(df_potential_tech2['WP_Term'].tolist())].tolist()

print (len(df_potential_tech1['WP_Term'].tolist() + df_potential_tech3['WP_Term'].tolist()))
print (len(tech_term_list))

3536
3442


# Wikidata Info Collection

In [4]:
import requests

def get_wikidata_id(wikipedia_article_name, lang='en'):
    """Fetch the Wikidata ID for a given Wikipedia article name."""
    url = f'https://{lang}.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'format': 'json',
        'titles': wikipedia_article_name,
        'prop': 'pageprops',
    }
    response = requests.get(url, params=params).json()
    page = next(iter(response['query']['pages'].values()))
    return page.get('pageprops', {}).get('wikibase_item')

def get_wikidata_info(wikidata_id):
    """Fetch Wikidata information using the Wikidata ID."""
    url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbgetentities',
        'ids': wikidata_id,
        'format': 'json',
        'languages': 'en',  # Adjust if you need information in another language
        'props': 'labels|descriptions|claims',  # Adjust based on the needed information
    }
    response = requests.get(url, params=params).json()
    return response['entities'][wikidata_id]



## Wikidata ID

In [32]:
article_id_dict = {}
index = 0

In [33]:
while index < len(tech_term_list):
    print (index)
    try:    
        article = tech_term_list[index]
        wikidata_id = get_wikidata_id(article)
        article_id_dict[article] = wikidata_id
        index = index + 1
    except KeyError as ke:
        print ('KeyError')
        print (article)
        index = index + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [34]:
df_wk_id = pd.DataFrame.from_dict(article_id_dict, orient = 'index')
df_wk_id = df_wk_id.reset_index()
df_wk_id.columns = ['Technology','Wikidata_ID']
df_wk_id['Technology'] = df_wk_id['Technology'].apply(lambda x: unquote(x))

## Missing Wikidata ID

In [53]:
df_missing = df_wk_id[df_wk_id['Wikidata_ID'].isna()]

missing_tech_list = df_missing['Technology'].tolist()

tech_index = 0
missing_id_dict = {}


In [54]:
while tech_index < len(missing_tech_list):
    print (tech_index)
    tech = missing_tech_list[tech_index]
    try:
        new_tech = wikipedia.search(tech)[0]
        wikidata_id = get_wikidata_id(new_tech)
        missing_id_dict[tech] = wikidata_id
        tech_index = tech_index + 1
    except IndexError:
        tech_index = tech_index + 1
    except wikipedia.exceptions.WikipediaException:
        tech_index = tech_index + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111


In [55]:
df_wk_missing_id = pd.DataFrame.from_dict(missing_id_dict, orient = 'index')
df_wk_missing_id = df_wk_missing_id.reset_index()
df_wk_missing_id.columns = ['Technology','Wikidata_ID']

df_wk_missing_id.loc[df_wk_missing_id['Technology']=='Switching circuit theory','Wikidata_ID'] = 'Q7659118'
df_wk_missing_id.loc[df_wk_missing_id['Technology']=='CoolSculpting','Wikidata_ID'] = 'Q71357346'
df_wk_missing_id.loc[df_wk_missing_id['Technology']=='Accessible QR','Wikidata_ID'] = 'Q12203'
df_wk_missing_id = df_wk_missing_id[~df_wk_missing_id['Wikidata_ID'].isna()]

In [62]:
df_wk_id[~df_wk_id['Wikidata_ID'].isna()]

,Technology,Wikidata_ID
0,3D optical data storage,Q2024185
1,4D printing,Q30324048
3,Active structure,Q4677591
5,Aerogel,Q381490
6,Agricultural robot,Q4414810
...,...,...
3437,In situ thermal desorption,Q6008667
3438,New Technologies Demonstrator Programme,Q7012055
3439,Fluorescent lamp crusher,Q5462826
3440,Scottish Landfill Tax,Q19059551


In [63]:
df_wk_id_all = pd.concat([df_wk_id[~df_wk_id['Wikidata_ID'].isna()],df_wk_missing_id])
print (df_wk_id_all.shape)
df_wk_id_all = df_wk_id_all.drop_duplicates()
print (df_wk_id_all.shape)


(3440, 2)
(3440, 2)


## Wikidata Info

In [88]:
df_potential_tech2[df_potential_tech2['Wikidata_ID'].isin(df_wk_id_all['Wikidata_ID'])].shape

(44, 6)

In [86]:
df_wk_id_all['Wikidata_ID'].isin()

,Technology,Wikidata_ID
0,3D optical data storage,Q2024185
1,4D printing,Q30324048
3,Active structure,Q4677591
5,Aerogel,Q381490
6,Agricultural robot,Q4414810
...,...,...
107,"International Union for Vacuum Science, Techni...",Q30632426
108,Folding@home,Q386283
109,Dialog+,Q5270580
110,Oil–water separator,Q10609181


In [ ]:
df_potential_tech2['Wikidata_ID'].tolist()

In [89]:
wikidata_id_list = df_wk_id_all[~df_wk_id_all['Wikidata_ID'].isna()]['Wikidata_ID'].tolist() + df_potential_tech2['Wikidata_ID'].tolist()


In [90]:
len(wikidata_id_list)

5941

In [91]:
len(list(set(wikidata_id_list)))

5870

In [92]:
wikidata_id_list = list(set(wikidata_id_list))

In [93]:
id_index = 0
row_list = []
claims_index = {"instance of":'P31', "subclass of":'P279', "Commons category":'P373', 
                "topic's main category":'P910', "Freebase ID":'P646', "Microsoft Academic ID":'P6366',
                "OpenAlex ID":'P10283'}

In [94]:
while id_index < len(wikidata_id_list):
    print (id_index)
    wikidata_id = wikidata_id_list[id_index]
    print (wikidata_id)
    wikidata_info = get_wikidata_info(wikidata_id)
    row_dict = {}
    row_dict['Wikidata_ID'] = wikidata_id
    if len(wikidata_info['descriptions']) != 0:
        row_dict['Description'] = wikidata_info['descriptions']['en']['value']
    else:
        row_dict['Description'] = np.nan
    for key, value in claims_index.items():
        if (value == 'P373') or (value == 'P646') or (value == 'P6366') or (value == 'P10283'):
            if value in list(wikidata_info['claims'].keys()):
                try:
                    row_dict[key] = ','.join([i['mainsnak']['datavalue']['value'] for i in wikidata_info['claims'][value]])
                except KeyError as ke:
                    row_dict[key] = np.nan
            else:
                row_dict[key] = np.nan
        else:
            if value in list(wikidata_info['claims'].keys()):
                try:
                    row_dict[key] = ','.join([i['mainsnak']['datavalue']['value']['id'] for i in wikidata_info['claims'][value]])
                except KeyError as ke:
                    row_dict[key] = np.nan
            else:
                row_dict[key] = np.nan
    row_list.append(row_dict)
    id_index = id_index + 1

0
Q6907031
1
Q7333030
2
Q968591
3
Q19715435
4
Q7398629
5
Q8055236
6
Q7631670
7
Q113297895
8
Q26856022
9
Q5972003
10
Q6944885
11
Q110262540
12
Q79289672
13
Q22052437
14
Q1075992
15
Q11384
16
Q12331276
17
Q5189486
18
Q7696770
19
Q4633137
20
Q2499543
21
Q107168929
22
Q67076196
23
Q25305187
24
Q116169291
25
Q6158790
26
Q19059551
27
Q122526192
28
Q5190528
29
Q549037
30
Q111169553
31
Q7696888
32
Q17077284
33
Q7426100
34
Q4833678
35
Q1348106
36
Q110262525
37
Q3298493
38
Q54366672
39
Q16001545
40
Q51617211
41
Q19599451
42
Q2073007
43
Q3025061
44
Q213901
45
Q25104872
46
Q25051577
47
Q3046590
48
Q5548572
49
Q5275718
50
Q110262477
51
Q1530371
52
Q4867810
53
Q5126353
54
Q189131
55
Q600568
56
Q25053039
57
Q1224643
58
Q7514077
59
Q6920832
60
Q15837622
61
Q7193142
62
Q55080168
63
Q993598
64
Q1224345
65
Q22058455
66
Q7068181
67
Q16886916
68
Q7975356
69
Q1348859
70
Q5445134
71
Q1324107
72
Q919226
73
Q1027896
74
Q17322072
75
Q5972915
76
Q110953036
77
Q114470482
78
Q4455336
79
Q3306798
80
Q123010405
81
Q

621
Q115995136
622
Q22000614
623
Q352072
624
Q5392830
625
Q1063710
626
Q287333
627
Q6964101
628
Q93482314
629
Q2565637
630
Q245471
631
Q953582
632
Q2302333
633
Q760102
634
Q19878318
635
Q15967034
636
Q1640383
637
Q176555
638
Q4086886
639
Q7839857
640
Q5348471
641
Q17460813
642
Q105813780
643
Q10588266
644
Q2091188
645
Q5264579
646
Q25326661
647
Q39052812
648
Q3391185
649
Q116169303
650
Q3538629
651
Q7825074
652
Q21467923
653
Q18351515
654
Q2375766
655
Q11697387
656
Q6749574
657
Q120422790
658
Q523483
659
Q2873
660
Q18212263
661
Q18360045
662
Q1779048
663
Q117831805
664
Q3152339
665
Q180944
666
Q2025366
667
Q4218491
668
Q29346036
669
Q7907466
670
Q173451
671
Q7281161
672
Q353268
673
Q55761015
674
Q2608905
675
Q6987428
676
Q60884818
677
Q16874768
678
Q1127694
679
Q21592150
680
Q1416880
681
Q76184166
682
Q17029687
683
Q5357738
684
Q1635630
685
Q2064646
686
Q7672094
687
Q543281
688
Q4340379
689
Q7621738
690
Q5324920
691
Q1413600
692
Q2982767
693
Q477227
694
Q611666
695
Q1881103
696
Q108761

1220
Q7696502
1221
Q23581542
1222
Q635115
1223
Q1125947
1224
Q93128616
1225
Q4048797
1226
Q1896121
1227
Q5149097
1228
Q28324866
1229
Q7016490
1230
Q1376247
1231
Q11193326
1232
Q62631415
1233
Q1474667
1234
Q488684
1235
Q117831829
1236
Q97597706
1237
Q28068773
1238
Q1075599
1239
Q2551748
1240
Q7236022
1241
Q124846095
1242
Q5008300
1243
Q4044991
1244
Q110262838
1245
Q8041813
1246
Q109435160
1247
Q966781
1248
Q62664512
1249
Q950502
1250
Q85867267
1251
Q48975016
1252
Q109535147
1253
Q177023
1254
Q104443514
1255
Q117857257
1256
Q4389835
1257
Q23679987
1258
Q9184145
1259
Q6639706
1260
Q4915399
1261
Q124328061
1262
Q669700
1263
Q1945033
1264
Q97971374
1265
Q1061598
1266
Q6714217
1267
Q1002972
1268
Q366979
1269
Q115014694
1270
Q19903839
1271
Q4536303
1272
Q929429
1273
Q15081379
1274
Q17155063
1275
Q905846
1276
Q5062163
1277
Q122443308
1278
Q117831951
1279
Q6840506
1280
Q1757851
1281
Q6510662
1282
Q753552
1283
Q7936590
1284
Q1127628
1285
Q62662754
1286
Q65038574
1287
Q9941
1288
Q106046678
1289
Q

1793
Q173198
1794
Q5416061
1795
Q5150819
1796
Q110989948
1797
Q5420744
1798
Q7122895
1799
Q633613
1800
Q5358122
1801
Q11837678
1802
Q110950529
1803
Q3780321
1804
Q25339557
1805
Q1125608
1806
Q5358521
1807
Q3492666
1808
Q5009593
1809
Q21165502
1810
Q3276547
1811
Q1405450
1812
Q124435283
1813
Q2823869
1814
Q2044571
1815
Q4927095
1816
Q335828
1817
Q7600576
1818
Q1360316
1819
Q7134077
1820
Q7658716
1821
Q64559508
1822
Q3859869
1823
Q163117
1824
Q1956679
1825
Q5384030
1826
Q674564
1827
Q7095968
1828
Q124619952
1829
Q85865228
1830
Q4438787
1831
Q627924
1832
Q54825599
1833
Q85795260
1834
Q17028154
1835
Q4312884
1836
Q2741258
1837
Q117841722
1838
Q7551227
1839
Q1660060
1840
Q40887743
1841
Q668500
1842
Q30672747
1843
Q108540635
1844
Q4645698
1845
Q877806
1846
Q60691066
1847
Q24331674
1848
Q229370
1849
Q232914
1850
Q7623282
1851
Q4837013
1852
Q5204571
1853
Q5968901
1854
Q123643871
1855
Q124635356
1856
Q13099273
1857
Q2902493
1858
Q431545
1859
Q210816
1860
Q87057712
1861
Q85812629
1862
Q370563
18

2365
Q5957991
2366
Q7435319
2367
Q574751
2368
Q28406466
2369
Q124022429
2370
Q794536
2371
Q478859
2372
Q18000022
2373
Q7232988
2374
Q1153902
2375
Q827841
2376
Q110262523
2377
Q1366077
2378
Q86850
2379
Q17131164
2380
Q2518117
2381
Q407560
2382
Q11743787
2383
Q123742170
2384
Q6718566
2385
Q17486688
2386
Q17105111
2387
Q87353251
2388
Q7648839
2389
Q111271838
2390
Q117831898
2391
Q62625259
2392
Q1425307
2393
Q16927658
2394
Q42417235
2395
Q85790788
2396
Q25378005
2397
Q7000848
2398
Q1029151
2399
Q127956
2400
Q2118214
2401
Q1983108
2402
Q647462
2403
Q61941624
2404
Q108456477
2405
Q1431974
2406
Q18811569
2407
Q6786269
2408
Q110262495
2409
Q28408352
2410
Q17010029
2411
Q4972091
2412
Q7862866
2413
Q30632426
2414
Q5167449
2415
Q2291875
2416
Q408483
2417
Q3270361
2418
Q13049686
2419
Q1075680
2420
Q3869436
2421
Q20098
2422
Q2991990
2423
Q17118209
2424
Q2566998
2425
Q1069646
2426
Q251212
2427
Q770289
2428
Q5462059
2429
Q5435991
2430
Q111621272
2431
Q7978191
2432
Q78850600
2433
Q7239636
2434
Q739209

...

3510
Q4115252
3511
Q2876763
3512
Q7199319
3513
Q110262609
3514
Q47012798
3515
Q206658
3516
Q5140123
3517
Q4781514
3518
Q3240002
3519
Q355293
3520
Q3392614
3521
Q587646
3522
Q123643872
3523
Q4011361
3524
Q10605254
3525
Q798548
3526
Q2192779
3527
Q7562515
3528
Q6484283
3529
Q1778563
3530
Q90405586
3531
Q2045252
3532
Q5972247
3533
Q8563761
3534
Q10602358
3535
Q280386
3536
Q3851913
3537
Q816441
3538
Q703656
3539
Q17098352
3540
Q63615183
3541
Q869010
3542
Q4955770
3543
Q110262483
3544
Q394011
3545
Q1454150
3546
Q110949352
3547
Q26218061
3548
Q16685834
3549
Q25324110
3550
Q62635390
3551
Q7072879
3552
Q861170
3553
Q1055226
3554
Q25052117
3555
Q12607578
3556
Q178048
3557
Q868705
3558
Q3875172
3559
Q1568577
3560
Q127995
3561
Q7300802
3562
Q20819854
3563
Q30589394
3564
Q5206399
3565
Q2578851
3566
Q795690
3567
Q754523
3568
Q114809644
3569
Q109364652
3570
Q97119772
3571
Q123998175
3572
Q7672466
3573
Q7187314
3574
Q107996555
3575
Q386369
3576
Q2427493
3577
Q123485776
3578
Q4839662
3579
Q5302767
358

4081
Q138845
4082
Q7924635
4083
Q229390
4084
Q110262549
4085
Q111364902
4086
Q1134090
4087
Q103820061
4088
Q5264326
4089
Q2074906
4090
Q11902450
4091
Q3892079
4092
Q2424501
4093
Q104534
4094
Q5281210
4095
Q1443081
4096
Q1875963
4097
Q16954846
4098
Q7239201
4099
Q2442367
4100
Q21661921
4101
Q124354127
4102
Q52768557
4103
Q1893753
4104
Q267166
4105
Q28130161
4106
Q122924071
4107
Q1912138
4108
Q17460780
4109
Q43035
4110
Q16251994
4111
Q110779994
4112
Q112541771
4113
Q111288852
4114
Q57515336
4115
Q365870
4116
Q2309907
4117
Q1533033
4118
Q115332277
4119
Q111961620
4120
Q7935062
4121
Q5251761
4122
Q12246850
4123
Q1196964
4124
Q12858436
4125
Q185091
4126
Q1192011
4127
Q3833047
4128
Q1126970
4129
Q4795982
4130
Q18388724
4131
Q28134000
4132
Q112112739
4133
Q7081473
4134
Q4292133
4135
Q6964175
4136
Q20147
4137
Q936407
4138
Q81235
4139
Q97359001
4140
Q6025174
4141
Q14777816
4142
Q1426666
4143
Q2252764
4144
Q96739201
4145
Q3394071
4146
Q12099190
4147
Q600121
4148
Q1499254
4149
Q770692
4150
Q11026

4651
Q7828626
4652
Q547283
4653
Q1302677
4654
Q162869
4655
Q18624145
4656
Q4951582
4657
Q930144
4658
Q118583118
4659
Q18206966
4660
Q56228346
4661
Q125168009
4662
Q5380811
4663
Q3625597
4664
Q7690474
4665
Q15107837
4666
Q5154133
4667
Q10854783
4668
Q25323716
4669
Q3359571
4670
Q5276052
4671
Q4915379
4672
Q418722
4673
Q831468
4674
Q11201325
4675
Q6167731
4676
Q108857033
4677
Q19903989
4678
Q5477480
4679
Q4202272
4680
Q1256781
4681
Q19385011
4682
Q3467747
4683
Q13424796
4684
Q2826075
4685
Q7692637
4686
Q18356139
4687
Q7915922
4688
Q3407025
4689
Q123643959
4690
Q667334
4691
Q7556699
4692
Q2023882
4693
Q11290562
4694
Q184199
4695
Q3954107
4696
Q19599382
4697
Q65088314
4698
Q110927977
4699
Q58181
4700
Q5508201
4701
Q46346792
4702
Q1139372
4703
Q17143668
4704
Q30525109
4705
Q5021475
4706
Q486936
4707
Q2173854
4708
Q2370416
4709
Q110262576
4710
Q11551596
4711
Q5321514
4712
Q1667753
4713
Q7269088
4714
Q7824635
4715
Q10389417
4716
Q33038478
4717
Q117831923
4718
Q30324048
4719
Q21057279
4720
Q21

5222
Q613680
5223
Q5449146
5224
Q2723514
5225
Q112482954
5226
Q1146414
5227
Q1661860
5228
Q37828
5229
Q1824681
5230
Q25521504
5231
Q15712216
5232
Q106641580
5233
Q104844004
5234
Q109280904
5235
Q1813564
5236
Q5527010
5237
Q7460752
5238
Q5282787
5239
Q4445012
5240
Q2165575
5241
Q587403
5242
Q2903140
5243
Q85750870
5244
Q1755670
5245
Q99680791
5246
Q110262478
5247
Q2450880
5248
Q4681351
5249
Q11950728
5250
Q2091479
5251
Q79289415
5252
Q864323
5253
Q4785259
5254
Q7299962
5255
Q488986
5256
Q11339323
5257
Q105845527
5258
Q609765
5259
Q118869775
5260
Q28756357
5261
Q4904834
5262
Q51832
5263
Q17011234
5264
Q28016156
5265
Q1369235
5266
Q830527
5267
Q28016162
5268
Q116788528
5269
Q7834120
5270
Q605108
5271
Q5356536
5272
Q115040618
5273
Q120422788
5274
Q107598383
5275
Q104880181
5276
Q123643957
5277
Q7832483
5278
Q116044716
5279
Q300113
5280
Q6980662
5281
Q841183
5282
Q3415048
5283
Q1754803
5284
Q1544518
5285
Q6495873
5286
Q15931103
5287
Q1291673
5288
Q2144090
5289
Q112079671
5290
Q1164091
5291


5795
Q110262607
5796
Q105972972
5797
Q5320697
5798
Q431344
5799
Q1563007
5800
Q1459101
5801
Q190434
5802
Q7972811
5803
Q1778158
5804
Q111190259
5805
Q5130055
5806
Q3683600
5807
Q643179
5808
Q7959428
5809
Q121614464
5810
Q1894866
5811
Q117455347
5812
Q16847353
5813
Q812656
5814
Q108102034
5815
Q120422792
5816
Q17165512
5817
Q15718547
5818
Q111189176
5819
Q16209135
5820
Q4228437
5821
Q15710541
5822
Q1373429
5823
Q106762644
5824
Q110262566
5825
Q56669074
5826
Q3398905
5827
Q3459505
5828
Q3928682
5829
Q2337129
5830
Q64399574
5831
Q5377938
5832
Q958936
5833
Q3597955
5834
Q109501195
5835
Q490249
5836
Q110917298
5837
Q836005
5838
Q742085
5839
Q20888462
5840
Q588750
5841
Q104835352
5842
Q106515723
5843
Q688498
5844
Q121130102
5845
Q104519230
5846
Q2835802
5847
Q51690493
5848
Q11189222
5849
Q5013338
5850
Q19582366
5851
Q120011964
5852
Q1154312
5853
Q17492657
5854
Q360071
5855
Q85811259
5856
Q65533608
5857
Q106515669
5858
Q2493
5859
Q5972109
5860
Q16994277
5861
Q1144899
5862
Q628904
5863
Q134166

In [95]:
df_wikidata_info = pd.DataFrame(row_list)

## Wikidata Item Name

In [96]:
instance_of_all_list = df_wikidata_info[~df_wikidata_info['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()

instance_of_list = [y for x in instance_of_all_list for y in x]

instance_unique_list = pd.Series(instance_of_list).value_counts().index.tolist()

print (len(instance_unique_list))

instance_index = 0
instance_of_row_list = []


1391


In [97]:
while instance_index < len(instance_unique_list):
    print (instance_index)
    wikidata_id = instance_unique_list[instance_index]
    row_dict = {}
    row_dict['Wikidata_ID'] = wikidata_id
    wikidata_info = get_wikidata_info(wikidata_id)
    try:
        row_dict['instance_of_label'] = wikidata_info['labels']['en']['value']
    except KeyError:
        row_dict['instance_of_label'] = np.nan
    instance_of_row_list.append(row_dict)
    instance_index = instance_index + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [98]:
df_instance_of_label = pd.DataFrame(instance_of_row_list)

In [99]:
df_instance_of_label

,Wikidata_ID,instance_of_label
0,Q11016,technology
1,Q110256698,Stack Exchange forum site
2,Q4830453,business
3,Q341,free software
4,Q17028492,hypothetical technology
...,...,...
1386,Q5830907,computer memory
1387,Q1872885,quantum effect
1388,Q131212,text editor
1389,Q15976092,artificial intelligence researcher


In [100]:
subclass_of_all_list = df_wikidata_info[~df_wikidata_info['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()

subclass_of_list = [y for x in subclass_of_all_list for y in x]

subclass_unique_list = pd.Series(subclass_of_list).value_counts().index.tolist()

print (len(subclass_unique_list))

subclass_index = 0
subclass_of_row_list = []

1489


In [101]:
while subclass_index < len(subclass_unique_list):
    print (subclass_index)
    wikidata_id = subclass_unique_list[subclass_index]
    row_dict = {}
    row_dict['Wikidata_ID'] = wikidata_id
    wikidata_info = get_wikidata_info(wikidata_id)
    try:
        row_dict['subclass_of_label'] = wikidata_info['labels']['en']['value']
    except KeyError:
        row_dict['subclass_of_label'] = np.nan
    subclass_of_row_list.append(row_dict)
    subclass_index = subclass_index + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [102]:
df_subclass_of_label = pd.DataFrame(subclass_of_row_list)

In [103]:
df_subclass_of_label

,Wikidata_ID,subclass_of_label
0,Q1183543,appliance
1,Q11016,technology
2,Q11468,nanotechnology
3,Q260899,shot
4,Q11425,animation
...,...,...
1484,Q428163,technical diving
1485,Q732942,video tape recorder
1486,Q16644043,diagnosis
1487,Q31728,public administration


# Tech Matrix

In [208]:
df_wiki_potential_tech1 = df_potential_tech1.copy()
df_wiki_potential_tech1['WP_Term'] = df_wiki_potential_tech1['WP_Term'].apply(lambda x: unquote(x))
df_wiki_potential_tech1 = df_wiki_potential_tech1.merge(df_wk_id_all.rename(columns = {'Technology':'WP_Term'}), how = 'left', left_on = 'WP_Term', right_on = 'WP_Term')
df_wiki_potential_tech2 = df_potential_tech2.copy()
df_wiki_potential_tech3 = df_potential_tech3.copy()
df_wiki_potential_tech3['WP_Term'] = df_wiki_potential_tech3['WP_Term'].apply(lambda x: unquote(x))
df_wiki_potential_tech3 = df_wiki_potential_tech3.merge(df_wk_id_all.rename(columns = {'Technology':'WP_Term'}), how = 'left', left_on = 'WP_Term', right_on = 'WP_Term')

df_wiki_potential_tech1['Category'] = 'List 1'
df_wiki_potential_tech2['Category'] = 'List 2'
df_wiki_potential_tech3['Category'] = 'List 3'

df_wiki_potential_tech1.rename(columns = {'Emerging Tech List (n=242)':'Wikipedia_Article'}, inplace = True)
df_wiki_potential_tech2.rename(columns = {'article':'Wikipedia_Article'}, inplace = True)
df_wiki_potential_tech3.rename(columns = {'WP Article':'Wikipedia_Article'}, inplace = True)

df_new_tech_lists = pd.concat([df_wiki_potential_tech1[['Wikipedia_Article','WP_Term','Wikidata_ID','Category']], 
                               df_wiki_potential_tech2[['Wikipedia_Article','WP_Term','Wikidata_ID','Category']],
                               df_wiki_potential_tech3[['Wikipedia_Article','WP_Term','Wikidata_ID','Category']]])
df_new_tech_lists = df_new_tech_lists.merge(df_wikidata_info, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

df_keep_tech1 = df_new_tech_lists[df_new_tech_lists['Category'] == 'List 1']
keep_instance_of1 = df_keep_tech1[~df_keep_tech1['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()
keep_instance_of_list1 = [y for x in keep_instance_of1 for y in x]
df_keep_instance_counts1 = pd.DataFrame(pd.Series(keep_instance_of_list1).value_counts()).reset_index()
df_keep_instance_counts1.columns = ['Wikidata_ID','List1_Counts']
df_keep_instance_counts1 = df_keep_instance_counts1.merge(df_instance_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

keep_subclass_of1 = df_keep_tech1[~df_keep_tech1['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()
keep_subclass_of_list1 = [y for x in keep_subclass_of1 for y in x]
df_keep_subclass_counts1 = pd.DataFrame(pd.Series(keep_subclass_of_list1).value_counts()).reset_index()
df_keep_subclass_counts1.columns = ['Wikidata_ID','List1_Counts']
df_keep_subclass_counts1 = df_keep_subclass_counts1.merge(df_subclass_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')



df_keep_tech2 = df_new_tech_lists[df_new_tech_lists['Category'] == 'List 2']
keep_instance_of2 = df_keep_tech2[~df_keep_tech2['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()
keep_instance_of_list2 = [y for x in keep_instance_of2 for y in x]
df_keep_instance_counts2 = pd.DataFrame(pd.Series(keep_instance_of_list2).value_counts()).reset_index()
df_keep_instance_counts2.columns = ['Wikidata_ID','List2_Counts']
df_keep_instance_counts2 = df_keep_instance_counts2.merge(df_instance_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

keep_subclass_of2 = df_keep_tech2[~df_keep_tech2['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()
keep_subclass_of_list2 = [y for x in keep_subclass_of2 for y in x]
df_keep_subclass_counts2 = pd.DataFrame(pd.Series(keep_subclass_of_list2).value_counts()).reset_index()
df_keep_subclass_counts2.columns = ['Wikidata_ID','List2_Counts']
df_keep_subclass_counts2 = df_keep_subclass_counts2.merge(df_subclass_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')



df_keep_tech3 = df_new_tech_lists[df_new_tech_lists['Category'] == 'List 3']
keep_instance_of3 = df_keep_tech3[~df_keep_tech3['instance of'].isna()]['instance of'].apply(lambda x: x.split(',')).tolist()
keep_instance_of_list3 = [y for x in keep_instance_of3 for y in x]
df_keep_instance_counts3 = pd.DataFrame(pd.Series(keep_instance_of_list3).value_counts()).reset_index()
df_keep_instance_counts3.columns = ['Wikidata_ID','List3_Counts']
df_keep_instance_counts3 = df_keep_instance_counts3.merge(df_instance_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')

keep_subclass_of3 = df_keep_tech3[~df_keep_tech3['subclass of'].isna()]['subclass of'].apply(lambda x: x.split(',')).tolist()
keep_subclass_of_list3 = [y for x in keep_subclass_of3 for y in x]
df_keep_subclass_counts3 = pd.DataFrame(pd.Series(keep_subclass_of_list3).value_counts()).reset_index()
df_keep_subclass_counts3.columns = ['Wikidata_ID','List3_Counts']
df_keep_subclass_counts3 = df_keep_subclass_counts3.merge(df_subclass_of_label, how = 'left', left_on = 'Wikidata_ID', right_on = 'Wikidata_ID')



df_keep_instance_counts_new_list = df_keep_instance_counts1.merge(df_keep_instance_counts2, how = 'outer', left_on = ['Wikidata_ID','instance_of_label'], right_on = ['Wikidata_ID','instance_of_label'])
df_keep_instance_counts_new_list = df_keep_instance_counts_new_list.merge(df_keep_instance_counts3, how = 'outer', left_on = ['Wikidata_ID','instance_of_label'], right_on = ['Wikidata_ID','instance_of_label'])
df_keep_instance_counts_new_list = df_keep_instance_counts_new_list[['Wikidata_ID','instance_of_label','List1_Counts','List2_Counts','List3_Counts']]
df_keep_instance_counts_new_list['instance_of_label'] = df_keep_instance_counts_new_list['instance_of_label'].replace({'part':'part (piece)','wiki software':'collaborative wiki software'})

df_keep_subclass_counts_new_list = df_keep_subclass_counts1.merge(df_keep_subclass_counts2, how = 'outer', left_on = ['Wikidata_ID','subclass_of_label'], right_on = ['Wikidata_ID','subclass_of_label'])
df_keep_subclass_counts_new_list = df_keep_subclass_counts_new_list.merge(df_keep_subclass_counts3, how = 'outer', left_on = ['Wikidata_ID','subclass_of_label'], right_on = ['Wikidata_ID','subclass_of_label'])
df_keep_subclass_counts_new_list = df_keep_subclass_counts_new_list[['Wikidata_ID','subclass_of_label','List1_Counts','List2_Counts','List3_Counts']]
df_keep_subclass_counts_new_list['subclass_of_label'] = df_keep_subclass_counts_new_list['subclass_of_label'].replace({'device app':'computer data storage device'})

df_et_all_instance_counts.to_csv('Datasets/ET50k_Wikidata_instance_of_label_counts.csv', index = False)
df_et_all_subclass_counts.to_csv('Datasets/ET50k_Wikidata_subclass_of_label_counts.csv', index = False)
df_instance_counts_comb = df_et_all_instance_counts.rename(columns = {'Counts':'ET50k_Counts'}).merge(df_keep_instance_counts_new_list, how = 'outer', left_on = ['Wikidata_ID','instance_of_label'], right_on = ['Wikidata_ID','instance_of_label'])
df_subclass_counts_comb = df_et_all_subclass_counts.rename(columns = {'Counts':'ET50k_Counts'}).merge(df_keep_subclass_counts_new_list, how = 'outer', left_on = ['Wikidata_ID','subclass_of_label'], right_on = ['Wikidata_ID','subclass_of_label'])

df_et100_instance_counts = pd.read_csv('Datasets/ET100_Wikidata_instance_of_label_counts.csv')
df_et100_subclass_counts = pd.read_csv('Datasets/ET100_Wikidata_subclass_of_label_counts.csv')
df_instance_counts_comb = df_instance_counts_comb.merge(df_et100_instance_counts.rename(columns = {'Counts':'ET100_Counts'}), how = 'outer', left_on = ['Wikidata_ID','instance_of_label'], right_on = ['Wikidata_ID','instance_of_label'])
df_subclass_counts_comb = df_subclass_counts_comb.merge(df_et100_subclass_counts.rename(columns = {'Counts':'ET100_Counts'}), how = 'outer', left_on = ['Wikidata_ID','subclass_of_label'], right_on = ['Wikidata_ID','subclass_of_label'])

df_instance_counts_comb = df_instance_counts_comb[['Wikidata_ID', 'instance_of_label', 'ET50k_Counts', 'ET100_Counts', 'List1_Counts', 'List2_Counts', 'List3_Counts']]
df_subclass_counts_comb = df_subclass_counts_comb[['Wikidata_ID', 'subclass_of_label', 'ET50k_Counts', 'ET100_Counts', 'List1_Counts', 'List2_Counts', 'List3_Counts']]

df_keep_instance = df_instance_counts_comb[(~df_instance_counts_comb['ET100_Counts'].isna()) | (~df_instance_counts_comb['List1_Counts'].isna()) | (~df_instance_counts_comb['List2_Counts'].isna()) | (~df_instance_counts_comb['List3_Counts'].isna())]
df_keep_subclass = df_subclass_counts_comb[(~df_subclass_counts_comb['ET100_Counts'].isna()) | (~df_subclass_counts_comb['List1_Counts'].isna()) | (~df_subclass_counts_comb['List2_Counts'].isna()) | (~df_subclass_counts_comb['List3_Counts'].isna())]



In [222]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter("Datasets/Wikidata_instance_sublcass_summary.xlsx", engine="xlsxwriter")

# Write each dataframe to a different worksheet.
df_keep_instance.to_excel(writer, sheet_name="instance summary", index = False)
df_keep_subclass.to_excel(writer, sheet_name="subclass summary", index = False)

# Close the Pandas Excel writer and output the Excel file.
writer.close()